In [1]:
import os
import papermill as pm
from IPython.display import display, HTML

def create_directory_if_not_exists(directory):
    """
    Ensures that the specified directory exists. If it doesn't, the directory is created.

    Parameters:
    directory (str): The path of the directory to check or create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)  # Create the directory if it doesn't exist
        print(f"Directory {directory} created.")

def run_notebook_with_parameters(input_path, output_path, params, processed_notebook_dir="./processed_notebook"):
    """
    Executes a single Jupyter notebook with specified parameters and saves the result
    in the processed notebooks directory.

    Parameters:
    input_path (str): The path to the input notebook to execute.
    output_path (str): The path to save the processed notebook result.
    params (dict): The parameters to pass to the notebook.
    processed_notebooks_dir (str): The directory where the processed notebooks will be saved.
    """
    # Ensure the processed notebooks directory exists
    create_directory_if_not_exists(processed_notebook_dir)

    print(f"Executing {input_path} with parameters ⚡ {params} ⚡ ...")
    try:
        # Execute the notebook with the provided parameters
        execution = pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            parameters=params
        )
        # Create a hyperlink for the result
        display(HTML(f"✅ Execution successful, check the result at the following link -> <a href='{output_path}' target='_blank'>{output_path}</a>"))
    except Exception as e:
        print(f"Error executing {input_path}: {e}")
        execution = None
        display(HTML(f"❌ Execution failed, check the error details and result at the following link -> <a href='{output_path}' target='_blank'>{output_path}</a>"))
    print()
    return execution

def display_notebook_variables_and_values(notebook_output_values, notebook_name):
    """
    Displays variable details extracted from a notebook.

    Args:
        notebook_output_values (dict): The extracted variable data from the notebook.
        notebook_name (str): The name of the notebook being processed.

    Returns:
        None
    """
    if notebook_output_values and isinstance(notebook_output_values, dict):
        print(f"The Notebook Name is {notebook_name}")
        
        # Iterate over each cell's data
        for key, value in notebook_output_values.items():
            print(f"⚓ The execution cell number is = {value['execution_cell_number']}")
            print(f"🌀 The operation in the cell is = {value['variable_operation']}")
            print(f"♻️ The variable cell name is = {value['variable_name']}")
            print(f"ℹ️ The result of the cell is = {value['variable_value']}")
            print()
    else:
        print("Skipping invalid or empty notebook entry.")
        print()
        
def extract_variable_data_from_notebook_cells(notebook_results):
    """
    Extracts variable data from the notebook cells and outputs the variable name,
    the operation (source), and the value associated with each variable.

    Args:
        notebook_results (dict): The notebook content in dictionary format.

    Returns:
        dict: A dictionary with cell execution count as keys and a dictionary of 
              'variable_operation', 'variable_name', and 'variable_value'.
    """
    output_values = {}

    # Ensure notebook_results is not None and contains the expected 'cells' key
    if notebook_results and isinstance(notebook_results, dict):
        for cell in notebook_results.get('cells', []):
            # Process only code cells with outputs
            if cell.get('cell_type') == 'code' and 'outputs' in cell:
                for output in cell.get('outputs', []):
                    # Check if the output contains 'text/plain' data
                    text_plain = output.get('data', {}).get('text/plain')
                    if text_plain:
                        source = cell.get('source', '').strip()
                        
                        # If there's an equals sign on the left side (variable assignment)
                        if "=" in source:
                            variable_names = [var.strip() for var in source.split("=")[0].split(",")]
                            values = [text_plain.strip()]

                            # If there are multiple variables on the left-hand side
                            if len(variable_names) > 1:
                                # Remove the first and last characters only if the right-hand side is a list or tuple
                                if text_plain.strip().startswith('[') or text_plain.strip().startswith('('):
                                    # Strip the first and last characters (i.e., the brackets or parentheses)
                                    values = text_plain.strip()[1:-1].split(',')
                                else:
                                    values = text_plain.strip().split(',')
                                values = [val.strip() for val in values]

                                # Store each variable with its corresponding value
                                for i, variable in enumerate(variable_names):
                                    key = f"cell_{cell.get('execution_count')}_{i + 1}"
                                    output_values[key] = {
                                        "execution_cell_number": f"cell_{cell.get('execution_count')}",
                                        "variable_operation": source,
                                        "variable_name": variable,
                                        "variable_value": values[i] if i < len(values) else None
                                    }
                            else:
                                # Handle a single variable assignment with a full value
                                output_values[f"cell_{cell.get('execution_count')}"] = {
                                    "execution_cell_number": f"cell_{cell.get('execution_count')}",
                                    "variable_operation": source,
                                    "variable_name": variable_names[0],
                                    "variable_value": values[0]
                                }
                        else:
                            # Handle the case when no variable assignment occurs
                            variable_name = source.split()[0] if source else None
                            output_values[f"cell_{cell.get('execution_count')}"] = {
                                "execution_cell_number": f"cell_{cell.get('execution_count')}",
                                "variable_operation": source,
                                "variable_name": variable_name,
                                "variable_value": text_plain
                            }
    return output_values

In [2]:
processed_directory = "./processed_notebook"

notebooks_with_parameters = [
    ("1_Add.ipynb", f"{processed_directory}/add_executed.ipynb", {"params": [10, 5, 7]}),
    ("4_Divide.ipynb", f"{processed_directory}/divide_executed.ipynb", {"x": 20, "y": 0}),
    ("2_Subtract.ipynb", f"{processed_directory}/subtract_executed.ipynb", {"x": 10, "y": 3}),
    ("3_Multiply.ipynb", f"{processed_directory}/multiply_executed.ipynb", {"inject_values": {'x': [2, 3], 'y': [4, 5]}}),
]

notebook_execution_results = list()

for input_path, output_path, params in notebooks_with_parameters:
    results = run_notebook_with_parameters(input_path, output_path, params)
    notebook_execution_results.append({"notebook_name":input_path, "notebook_output_data": results})

Passed unknown parameter: params


Executing 1_Add.ipynb with parameters ⚡ {'params': [10, 5, 7]} ⚡ ...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]


Executing 4_Divide.ipynb with parameters ⚡ {'x': 20, 'y': 0} ⚡ ...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

...


Executing 2_Subtract.ipynb with parameters ⚡ {'x': 10, 'y': 3} ⚡ ...


Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

Passed unknown parameter: inject_values



Executing 3_Multiply.ipynb with parameters ⚡ {'inject_values': {'x': [2, 3], 'y': [4, 5]}} ⚡ ...


Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

# OBTENER LOS VALORES DE LAS VARIABLES DE CADA NOTEBOOK

In [3]:
variable_data_from_notebook = list()

for notebook_execution in notebook_execution_results:
    notebook_name = notebook_execution.get('notebook_name')
    notebook_data = extract_variable_data_from_notebook_cells(notebook_execution.get('notebook_output_data'))
    variable_data_from_notebook.append({'notebook_name': notebook_name, 'notebook_data':notebook_data})

# VER EL VALOR DE LAS VARIABLES DE CADA NOTEBOOK

In [4]:
for variable_from_notebook in variable_data_from_notebook:
    notebook_name = variable_from_notebook.get('notebook_name')
    notebook_data = variable_from_notebook.get('notebook_data')
    display_notebook_variables_and_values(notebook_data, notebook_name)

The Notebook Name is 1_Add.ipynb
⚓ The execution cell number is = cell_3
🌀 The operation in the cell is = add(*params)
♻️ The variable cell name is = add(*params)
ℹ️ The result of the cell is = 22

Skipping invalid or empty notebook entry.

The Notebook Name is 2_Subtract.ipynb
⚓ The execution cell number is = cell_4
🌀 The operation in the cell is = subtract(x,y)
♻️ The variable cell name is = subtract(x,y)
ℹ️ The result of the cell is = 7

The Notebook Name is 3_Multiply.ipynb
⚓ The execution cell number is = cell_4
🌀 The operation in the cell is = multiply(x,y)
♻️ The variable cell name is = multiply(x,y)
ℹ️ The result of the cell is = 45

⚓ The execution cell number is = cell_5
🌀 The operation in the cell is = one = multiply(x,y)
one
♻️ The variable cell name is = one
ℹ️ The result of the cell is = 45

⚓ The execution cell number is = cell_6
🌀 The operation in the cell is = two = multiply(x,y) * 2
two
♻️ The variable cell name is = two
ℹ️ The result of the cell is = 90

⚓ The execut